In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
print(orders.columns)
print(users.columns)
print(restaurants.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')
Index(['user_id', 'name', 'city', 'membership'], dtype='object')
Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')


In [6]:
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [7]:
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [9]:
final_df.shape
final_df.isnull().sum()


order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [10]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [11]:
revenue_by_city = (
    final_df
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

revenue_by_city


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

In [12]:
avg_order_value = final_df["total_amount"].mean()
round(avg_order_value, 2)


np.float64(801.16)

In [13]:
gold_aov = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_aov, 2)


np.float64(797.15)

In [14]:
cuisine_revenue = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

cuisine_revenue


cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [15]:
cuisine_aov = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

round(cuisine_aov, 2)


cuisine
Mexican    808.02
Italian    799.45
Indian     798.47
Chinese    798.39
Name: total_amount, dtype: float64

In [16]:
cuisine_restaurants = (
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)

cuisine_restaurants


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [21]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [22]:
quarterly_revenue = (
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarterly_revenue


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64